In [38]:
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

# Chapter 11. Training Deep Neural Network 

### 깊은 DNN을 학습시킬 때 생길 수 있는 문제들 
#### - vanishing(exploding) gradient 
- 깊은 신경망에 영향을 미치는 기울기가 사라지는 문제 


#### - 대규모 네트워크에서 느려지는 학습 속도 
#### - 많은 파라미터가 있는 모델의 학습 데이터에 대한 overfitting 위험성 



### 이번 장에서는 
#### - vanishing gradient 문제를 설명하고 가장 대중적인 해결방법을 설명 
#### - 다양한 기울기 하강법을 비교하여 대용량 모델을 빠르게 학습할 수 있는 다양한 최적화 기법 
#### - 대규모 신경망에서 인기있는 정규화 기법 

## Vanishing / Exploding Gradients Problems
#### - 역전파 알고리즘 
- 출력 레이어에서 입력 레이어로 이동하면서 오차 기울기를 전파
    - 네트워크의 각 파라미터와 관련하여 비용함수의 기울기를 계산
    - 계산한 기울기를 Gradient Descent Step를 통해 각 파라미터를 업데이트
    


#### - vanishing gradient 문제 
- 역전파 알고리즘이 하위 레이어로 진행되면서 기울기는 점점 작아짐 
-  그러므로 업데이트가 거의 반영되지 않아 좋은 해결책으로 수렴되지 않음 


#### - exploding gradient 문제 
- 반대의 경우, 기울기가 점점 커짐 
- 많은 가중치가 대폭 업데이트되고 결국 알고리즘은 발산 
- RNN에서 주로 발생하는 문제 



#### - 원인 
- 시그모이드 활성화 함수와 
- 평균 0, 표준편차 1의 정규분포를 사용하여 무작위로 가중치를 초기화하는 초기화 기법의 조합 
    - 시그모이드 활성화 함수와 초기화 전략이 각 레이어에서 출력의 분산이 입력의 분산보다 훨씬 크다는 것을 보임 
    - 네트워크 순방향 진행시, 분산은 활성화 함수가 최상위 레이어에서 포화될 때까지 증가함 
        - 로지스틱 함수의 평균이 0이 아닌 0.5이기 때문에 더 악화됨 
   
#### - 시그모이드 활성화 함수 
<img src = "image\ch11\sigmoid.png">
- 입력 값이 커지면 함수가 0 또는 1로 포화되고 도함수(미분계수)가 0에 가까워짐 
- 그러므로 역전파가 진행될 때, 전파될 수 있는 기울기가 거의 없음 
- 최상위 레이어로부터 기울기가 전파되어 올 때, 기울기가 점점 희박해지면서 하위 레이어로 전파될 기울기가 남지 않음 


### Xavier and He Initialization 
#### - 우리는 신호가 양방향으로 잘 흘러갈 수 있도록 해야 함 
- 예측 시의 순방향 전달, 기울기 역전파 시의 역방향 전달 
    
#### - 신호가 적절하게 흐르기 위해서는 입력과 출력의 분산이 같아야함 
#### - 또한 역방향 시, 레이어를 통과하기 전과 후의 분산이 같아야 함 
- 레이어가 동일한 개수의 입력과 출력연결을 가질 것을 보장할 수 없지만 실제로 잘 작동하는 것으로 입증된 절충안을 제시함 

#### - Xavier initialization(Glorot initialization)
<img src = "image\ch11\xavier.png">
- 연결 가중치는 무작위로 초기화 됨 
- n_inputs, n_outputs : 가중치가 초기화되는 레이어에 대한 입력과 출력 연결의 개수 
- 입력과 출력의 개수가 대략 같을 경우 사용할 수 있는 방정식 
<img src = "image\ch11\xavier2.png">

- 이 전략을 사용하면 학습속도가 상당히 빨라짐 
- 다양한 활성화 함수에 대한 유사한 전략 
    - ReLU활성화 함수(변형 포함)에 대한 초기화 전략은 He 초기화 전략이라 부르기도 함 
<img src = "image\ch11\init.png">

#### - fully_connected() 함수는 기본적으로 균일분포(uniform distribution)인 Xavier 초기화 전략을 사용함 
- variance_scaling_initializer()함수를 사용하여 He initialization으로 변경 가능 

In [2]:
import tensorflow as tf 

n_inputs = 28 * 28  # MNIST
n_hidden1 = 300

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")

In [6]:
from tensorflow.contrib.layers import fully_connected

he_init = tf.contrib.layers.variance_scaling_initializer() #He 초기화전략 
hidden1 = fully_connected(X, n_hidden1, weights_initializer=he_init,
                          scope="h1")

### Nonsaturating Activation Functions 
#### - vanishing/exploding gradients 문제의 부분적인 원인은 활성화 함수를 잘못 선택했기 때문 
- 깨닫기 전까지, 대부분의 사람들은 시그모이드 함수가 최선의 선택이라 생각
- 하지만 DNN에서 다른 활성화 함수, 특히 ReLU활성화 함수가 더 잘 동작한다는 사실이 밝혀짐 
    - ReLU 활성화 함수는 양수 값에 대해 포화되지 않으며 계산 속도가 매우 빠름 
    
#### - ReLU 활성화 함수
<img src = "image\ch11\relu.png">

- dying ReLUs 문제
    - ReLU 활성화 함수는 0보다 작은 값에서 기울기 값은 0이 됨 
    - 학습 동안 몇몇의 뉴런은 실제적으로 죽게됨 (0 이외의 다른 값을 출력하지 않음) 
    - 경우에 따라 네트워크 뉴런의 절반이 죽어있을 수도 있음 (학습률이 클수록)

#### - 변형된 ReLU 활성화 함수 
##### - leaky ReLU 활성화 함수

<img src = "image\ch11\lrelu2.png">
<img src = "image\ch11\lrelu.png">
- 작은 경사면을 통해 dying relu 문제가 생기지 않도록 함 
- 파라미터 α는 함수가 얼마나 많이 'leaks'할 것인가를 정의함 
    - z < 0에 대한 함수의 기울기이며, 일반적으로 0.01로 설정됨 
    
- 실제로 strict ReLU보다 leaku ReLU가 더 좋은 성능을 보임 
- 또한 α = 0.2(huge leak)일 때가 α = 0.01(small leak)일 때보다 더 좋은 성능을 보임


##### - RReLU (Randomized leaky ReLU)
- 학습 동안 α 값을 주어진 범위에서 무작위로 추출하여 학습 
    - 테스트할 때는 평균값으로 고정 
   
   
- 잘 동작하며 regularizer의 역할을 수행하는 것 처럼 보임 
    - 학습데이터에대한 과적합 위험을 줄임 
    
    
##### - PReLU (Parametic leaky ReLU)
- α는 역전파에 의해 수정될 수 있는 파라미터 값으로 학습 됨 
- PReLU는 큰 이미지 데이터 셋에서 ReLU보다 더 잘 동작함 
- 하지만 작은 이미지 데이터 셋에서는 학습데이터가 과적합될 위험이 있음 


#### - 새로운 활성화 함수 
##### - ELU (Exponential Linear Unit) 
- 학습 시간을 단축시켰으며 테스트 셋에서 더 잘 수행됨 
<img src = "image\ch11\elu.png">

- ReLU와 ELU의 차이점 
    - z < 0, 음의 값을 취함 
        - 단위가 0에 가까운 평균 출력을 갖으면서 vanishing gradient 문제 완화에 도움이 됨 
        - 파라미터 α는 z가 큰 음수이 때 ELU 함수가 접근하는 값을 정의함 
        - 보통 1로 설정하지만 조정 가능 
    - z < 0, 기울기가 0이 아님 
        - dying units 문제를 피할 수 있음 
    - 함수는 z=0 주변을 포함하여 모든 면에서 부드러움(smooth)
        - gradinet descent의 속도를 높이는데에 도움이 됨 


- 단점 
    - 지수 함수를 사용하기 떄문에 ReLU와 ReLU 변형들 보다 속도가 느림 
    
    
#### ※
- 많은 활성화 함수가 존재, DNN의 은닉레이어에서 어떤 활성화 함수를 사용해야 할까?
    - 일반적으로 ELU > leaky ReLU(또는 그 변형) > ReLU > tanh > sigmoid 를 사용함 
        - 실행시간 성능이 중요하다면 ELU보다 leaky ReLU를 사용하는 것이 좋음 
    - 다른 하이퍼파라미터를 조정하고 싶지 않다면 기본값 α를 사용할 수 있음 
        - leaky ReLU는 0.01
        - ELU는 1 
    - 시간적 여유와 컴퓨팅 능력이 있다면 cross validation을 이용하여 다른 활성화 함수를 평가
        - 특히 네트워크가 과적합된다면 RReLU를, 학습데이터가 너무 많으면 PReLU 사용 

#### - 구현 
- tensorflow는 신경망 구축에 사용할 수 있는 elu()함수를 제공함 
- fully_connected()함수 호출 시 activation_fn 인자를 설정해주면 됨 

In [8]:
hidden1 = fully_connected(X, n_hidden1, activation_fn=None)

- tensorflow에 leaku ReLU에 대해 정의된 함수는 없지만 정의하는 것은 쉬움 

In [31]:
import numpy as np 

def leaky_relu(z, name=None):
    return tf.maximum(0.01*z, z, name=name) #α=0.01

In [32]:
hidden1 = fully_connected(X, n_hidden1, activation_fn=leaky_relu)

### Batch Normalization 
#### - vanishing/exploding gradients 문제 해겨를 위해 BN 기술 제안 
- 학습 동안, 이전 레이어의 파라미터가 변경되면서 각 레이어들의 입력분포가 변경되는 문제(Internal Covariate Shift) 해결 

#### - 각 레이어의 활성화 함수 직전에 추가적인 작업을 함 
- 입력 값들을 zero centering과 normalizing을 함 
- 레이어 다 두개의 새로운 파라미터를 사용해 결과를 scaling, shifting 함 
- 즉, 이 작업을 통해 모델은 각 레이어의 입력에 대해 최적의 크기와 평균을 학습할 수 있다 

#### - zero-center, normalization
- 입력들의 평균 및 표준편차를 추정해야함 
    - 현재 mini-batch를 통해 평균과 표준편차를 평가함 
    <img src = "image\ch11\bn.png">
   

- 테스트 시에는 경험적 평균과 표준편차를 계산할 mini-batch가 없으므로 대신 전체 학습 셋의 평균 및 표준편차를 사용함 
- 결론적으로 4개의 파라미터가 학습됨 
    - 스케일, 위치, 평균, 표준편차 
    
    
#### - 효과 
- 실험된 모든 DNN을 많이 개선하는데 도움 
- vanishing gradients 문제도 많이 개선시킴 
- 가중치 초기화에 덜 민감함 
    - 훨씬 더 큰 학습률을 사용할 수 있어 학습 속도를 크게 향상 시킴 
    
- regulaizer로 작동하여 dropout과 같은 정규화 기법의 필요성을 줄여줌 


#### - 단점 
- batch-normalized 상태 일때는 첫번재 은닉레이어가 이를 처리하기 떄문에 입력 데이터를 정규화할 필요는 없지만 모델에 약간의 복잡성이 추가됨 
- 런타임 패널티 
    - 신경망은 각 레이어에서 필요한 추가 계산으로 인해 예측이 느려짐 
    - 빠른 예측을 원한다면 BN 수행 전에 ELU와 He 초기화 전략이 제대로 수행되는지 확인 필요  
    
    
#### - Implementing Batch Normalization with Tensorflow 
- tensorflow에서 입력을 중앙에 놓고 정규화하는 함수를 제공
    - 하지만 사용자가 직접 평균 및 표준편차를 계산해야 함 
  
- 파라미터를 함수에 전달하고 scaling 및 offset 파라미터의 생성을 처리해야 함 
- 복잡하니까 
- batch_norm() 함수를 사용하면 됨 
    - fully_connected()를 바로 사용해도 됨 
   

In [42]:
reset_graph()

from tensorflow.contrib.layers import batch_norm 

n_inputs = 28 * 28
n_hidden1 = 300
n_hidden2 = 100
n_outputs = 10

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")

is_training = tf.placeholder(tf.bool, shape=(), name='is_training')
bn_params = {
    'is_training' : is_training,
    'decay' : 0.99,
    'updates_collections' : None
} 

hidden1 = fully_connected(X, n_hidden1, scope='hidden1', 
                         normalizer_fn=batch_norm, normalizer_params=bn_params)
hidden2 = fully_connected(hidden1, n_hidden2, scope='hidden2', 
                         normalizer_fn=batch_norm, normalizer_params=bn_params)
logits = fully_connected(hidden2, n_outputs, scope='outputs', 
                         normalizer_fn=batch_norm, normalizer_params=bn_params)

In [45]:
reset_graph()

from tensorflow.contrib.layers import batch_norm 

n_inputs = 28 * 28
n_hidden1 = 300
n_hidden2 = 100
n_outputs = 10

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")

is_training = tf.placeholder(tf.bool, shape=(), name='is_training')
bn_params = {
    'is_training' : is_training,
    'decay' : 0.99,
    'updates_collections' : None
} 

#간단하게 묶어서 표현 
with tf.contrib.framework.arg_scope(
    [fully_connected],
    normalizer_fn=batch_norm,
    normalizer_params=bn_params) :

    hidden1 = fully_connected(X, n_hidden1, scope='hidden1')
    hidden2 = fully_connected(hidden1, n_hidden2, scope='hidden2')
    logits = fully_connected(hidden2, n_outputs, scope='outputs')

### Gradient Clipping 
#### - exploding gradients 문제를 줄이기 위한 기법
#### - 역전파 동안 임계값을 초과하지 않도록 하기 위해 기울기를 오려내는 방법 

#### - tensorflow 구현 
- tensorflow에서 최적기(optimzaer)의 minimize()함수는 기울기 계산과 적용을 수행함 
    - 최적기의 compute_gradients()함수 호출 
    - clip_by_value()함수를 사용하여 기울기를 오리는 작업을 생성 
    - 최적기의 apply_gradients()함수를 ㅏ용하여 오려낸 기울기를 적용하는 작업을 생성 

In [49]:
reset_graph()

n_inputs = 28 * 28  # MNIST
n_hidden1 = 300
n_hidden2 = 50
n_hidden3 = 50
n_hidden4 = 50
n_hidden5 = 50
n_outputs = 10

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int64, shape=(None), name="y")

with tf.name_scope("dnn"):
    hidden1 = tf.layers.dense(X, n_hidden1, activation=tf.nn.relu, name="hidden1")
    hidden2 = tf.layers.dense(hidden1, n_hidden2, activation=tf.nn.relu, name="hidden2")
    hidden3 = tf.layers.dense(hidden2, n_hidden3, activation=tf.nn.relu, name="hidden3")
    hidden4 = tf.layers.dense(hidden3, n_hidden4, activation=tf.nn.relu, name="hidden4")
    hidden5 = tf.layers.dense(hidden4, n_hidden5, activation=tf.nn.relu, name="hidden5")
    logits = tf.layers.dense(hidden5, n_outputs, name="outputs")

with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss")

In [51]:
learning_rate = 0.01

threshold = 1.0

optimizer = tf.train.GradientDescentOptimizer(learning_rate)

grads_and_vars = optimizer.compute_gradients(loss)

capped_gvs = [(tf.clip_by_value(grad, -threshold, threshold), var)
              for grad, var in grads_and_vars]
#-1 ~ 1 영역을 남기고 오려냄 

training_op = optimizer.apply_gradients(capped_gvs)

## Reusing Pretained Layers 
#### 매우 큰 DNN을 처음부터 학습하는 것 보다는 진행하려는 것과 비슷한 작업을 수행하는 기존 신경망을 찾아서 그 신경망의 하위 계층을 재사용하는 것이 좋다 (trnasfer learning)
- 학습속도를 크기 높이며 훨씬 더 적은 학습데이터를 필요로 함 
- 예) 
    - 일상적인 물체들을 100가지 범주로 분류하도록 학습된 DNN이 있을 때, 
    - 특정 차량 유형을 분류하는 DNN을 학습시키고 싶다 
        - 유사한 작업임으로 기존의 DNN의 일부 재사용 가능 
          <img src = "image\ch11\reuse.png">
          
          
          
### Reusing a Tensorflow Model 
#### - 기존 모델이 tensorlow를 통해 학습된 것이라면 간단하게 복원하여 새 작업에 활용할 수 있음 
- 기존 모델의 일부분만 잘라내서 사용할 수 있음 
    - 이 경우, 사전 학습 된 모델을 복원하기 위해 Saver 및 새로운 모델을 저장하기 위한 다른 Saver를 필요로 함 
    - 아래 코드는 은닉레이어 1, 2, 3 만 복원함 
        - 기존 모델의 은닉레이어 1, 2, 3을 복사해서 새로운 모델을 생성 
        - 정규표현식을 사용하여 은닉레이어 1, 2, 3의 모든 변수 목록을 가져옴 
        - 기존 모델의 각 변수 이름을 새 모델의 이름으로 매핑하는 사전을 만듦 
        - 이러한 변수들만 복원할 Saver를 생성 
        - 모든 전체 변수를 초기화하는 작업을 생성 
        - 두번째 Saver는 모델 전체를 저장함 
        - 세션을 열어 모델의 모든 변수를 초기화한 후 기존 모델의 은닉레이어 1 ~3의 변수를 복원함 
        - 마지막으로 새로운 작업에 대한 모델을 학습하고 이를 저장함 
        
        
#### ※
- 작업이 비슷할 수록 더 많은 레이어를 재사용할 수 있음 
- 매우 비슷한 작업의 경우 모든 은닉레이어를 유지하고 출력 레이어만 바꿀 수도 있음 

In [58]:
reset_graph()

n_inputs = 28 * 28  # MNIST
n_hidden1 = 300
n_hidden2 = 100
n_outputs = 10

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int64, shape=(None), name="y")

with tf.name_scope("dnn"):
    hidden1 = tf.layers.dense(X, n_hidden1, activation=selu, name="hidden1")
    hidden2 = tf.layers.dense(hidden1, n_hidden2, activation=selu, name="hidden2")
    logits = tf.layers.dense(hidden2, n_outputs, name="outputs")

with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss")

learning_rate = 0.01

with tf.name_scope("train"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)

with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

init = tf.global_variables_initializer()
saver = tf.train.Saver()
n_epochs = 40
batch_size = 50

NameError: name 'selu' is not defined

In [59]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/")

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [61]:
n_epochs = 40
batch_size = 50

with tf.Session() as sess:
    saver.restore(sess, "./my_model_final.ckpt")
    for epoch in range(n_epochs):
        for iteration in range(mnist.train.num_examples // batch_size):
            X_batch, y_batch = mnist.train.next_batch(batch_size)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        if epoch % 5 == 0:
            acc_train = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
            acc_test = accuracy.eval(feed_dict={X: mnist.validation.images, y: mnist.validation.labels})
            print(epoch, "Batch accuracy:", acc_train, "Validation accuracy:", acc_test)

    save_path = saver.save(sess, "./my_model_final.ckpt")

INFO:tensorflow:Restoring parameters from ./my_model_final.ckpt


TypeError: Cannot interpret feed_dict key as Tensor: The name 'save/Const:0' refers to a Tensor which does not exist. The operation, 'save/Const', does not exist in the graph.

In [60]:
with tf.Session() as sess:
    saver.restore(sess, "./my_model_final.ckpt")

    for epoch in range(n_epochs):
        for iteration in range(mnist.train.num_examples // batch_size):
            X_batch, y_batch = mnist.train.next_batch(batch_size)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        accuracy_val = accuracy.eval(feed_dict={X: mnist.test.images,
                                                y: mnist.test.labels})
        print(epoch, "Test accuracy:", accuracy_val)

    save_path = saver.save(sess, "./my_new_model_final.ckpt")    

INFO:tensorflow:Restoring parameters from ./my_model_final.ckpt


TypeError: Cannot interpret feed_dict key as Tensor: The name 'save/Const:0' refers to a Tensor which does not exist. The operation, 'save/Const', does not exist in the graph.

### Reusing Models from Other Frameworks 
#### - 다른 프레임워크를 사용하여 학습된 기존 모델이라면, 수동적으로 모델의 파라미터를 가져와야 함 
- 그 후 해당 파라미터를 적절한 변수에 할당해야 함 

#### - tensorflow 구현 
- 다른 프레임워크를 이용하여 사전에 학습된 모델을 불러오고, 재사용하려는 모델 파라미터를 추출 
- 텐서플로우 모델 구축 
- 모든 텐서플로우 변수에는 해당 변수를 초기화하는데 사용하는 관련 할당 작업이 있음
    - 할당 작업(변수이름과 같고 /Assign을 덧붙임)을 처리하는 것으로 시작 
    - 각 

### Freezing the Lower Layers
#### - 기존 DNN의 하위 레이어가 이미지의 저차원 특징들 감지를 학습했다면 그 레이어를 그대로 재사용 할 수 있음 
- 새로운 DNN을 학습할 때 일반적으로 가중치를 동결하는 것이 좋음 
- 하위 레이어의 가중치가 고정된 경우, 상위 레이어의 가중치를 쉽게 조정할 수 있음 
- 하위 레이어를 고정하는 방법은 하위 레이어의 변수를 제외하고 최적화할 수 있는 변수 목록을 제공하는 것

- 그래프에 stop_gradient(레이어)를 추가하는 것  


In [62]:
reset_graph()

n_inputs = 28 * 28  # MNIST
n_hidden1 = 300 # reused
n_hidden2 = 50  # reused
n_hidden3 = 50  # reused
n_hidden4 = 20  # new!
n_outputs = 10  # new!

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int64, shape=(None), name="y")

with tf.name_scope("dnn"):
    hidden1 = tf.layers.dense(X, n_hidden1, activation=tf.nn.relu, name="hidden1")       # reused
    hidden2 = tf.layers.dense(hidden1, n_hidden2, activation=tf.nn.relu, name="hidden2") # reused
    hidden3 = tf.layers.dense(hidden2, n_hidden3, activation=tf.nn.relu, name="hidden3") # reused
    hidden4 = tf.layers.dense(hidden3, n_hidden4, activation=tf.nn.relu, name="hidden4") # new!
    logits = tf.layers.dense(hidden4, n_outputs, name="outputs")                         # new!

with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss")

with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name="accuracy")

In [63]:
with tf.name_scope("train"):                                         # not shown in the book
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)     # not shown
    train_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES,
                                   scope="hidden[34]|outputs")
    training_op = optimizer.minimize(loss, var_list=train_vars)

In [64]:
init = tf.global_variables_initializer()
new_saver = tf.train.Saver()

In [66]:
reuse_vars = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES,
                               scope="hidden[123]") # regular expression
reuse_vars_dict = dict([(var.op.name, var) for var in reuse_vars])
restore_saver = tf.train.Saver(reuse_vars_dict) # to restore layers 1-3

init = tf.global_variables_initializer()
saver = tf.train.Saver()

with tf.Session() as sess:
    init.run()
    restore_saver.restore(sess, "./my_model_final.ckpt")

    for epoch in range(n_epochs):
        for iteration in range(mnist.train.num_examples // batch_size):
            X_batch, y_batch = mnist.train.next_batch(batch_size)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        accuracy_val = accuracy.eval(feed_dict={X: mnist.test.images,
                                                y: mnist.test.labels})
        print(epoch, "Test accuracy:", accuracy_val)


INFO:tensorflow:Restoring parameters from ./my_model_final.ckpt


NotFoundError: Key hidden2/bias not found in checkpoint
	 [[Node: save_3/RestoreV2_2 = RestoreV2[dtypes=[DT_FLOAT], _device="/job:localhost/replica:0/task:0/device:CPU:0"](_arg_save_3/Const_0_0, save_3/RestoreV2_2/tensor_names, save_3/RestoreV2_2/shape_and_slices)]]

Caused by op 'save_3/RestoreV2_2', defined at:
  File "C:\Users\jsh\Anaconda3\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\jsh\Anaconda3\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\jsh\Anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\Users\jsh\Anaconda3\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "C:\Users\jsh\Anaconda3\lib\site-packages\ipykernel\kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
  File "C:\Users\jsh\Anaconda3\lib\site-packages\zmq\eventloop\ioloop.py", line 177, in start
    supe

In [67]:
reset_graph()

n_inputs = 28 * 28  # MNIST
n_hidden1 = 300 # reused
n_hidden2 = 50  # reused
n_hidden3 = 50  # reused
n_hidden4 = 20  # new!
n_outputs = 10  # new!

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int64, shape=(None), name="y")

In [68]:
with tf.name_scope("dnn"):
    hidden1 = tf.layers.dense(X, n_hidden1, activation=tf.nn.relu,
                              name="hidden1") # reused frozen
    hidden2 = tf.layers.dense(hidden1, n_hidden2, activation=tf.nn.relu,
                              name="hidden2") # reused frozen
    hidden2_stop = tf.stop_gradient(hidden2)
    hidden3 = tf.layers.dense(hidden2_stop, n_hidden3, activation=tf.nn.relu,
                              name="hidden3") # reused, not frozen
    hidden4 = tf.layers.dense(hidden3, n_hidden4, activation=tf.nn.relu,
                              name="hidden4") # new!
    logits = tf.layers.dense(hidden4, n_outputs, name="outputs") # new!

In [69]:
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss")

with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name="accuracy")

with tf.name_scope("train"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)

In [70]:
reuse_vars = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES,
                               scope="hidden[123]") # regular expression
reuse_vars_dict = dict([(var.op.name, var) for var in reuse_vars])
restore_saver = tf.train.Saver(reuse_vars_dict) # to restore layers 1-3

init = tf.global_variables_initializer()
saver = tf.train.Saver()

with tf.Session() as sess:
    init.run()
    restore_saver.restore(sess, "./my_model_final.ckpt")

    for epoch in range(n_epochs):
        for iteration in range(mnist.train.num_examples // batch_size):
            X_batch, y_batch = mnist.train.next_batch(batch_size)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        accuracy_val = accuracy.eval(feed_dict={X: mnist.test.images,
                                                y: mnist.test.labels})
        print(epoch, "Test accuracy:", accuracy_val)

    save_path = saver.save(sess, "./my_new_model_final.ckpt")

INFO:tensorflow:Restoring parameters from ./my_model_final.ckpt


NotFoundError: Key hidden1/bias not found in checkpoint
	 [[Node: save/RestoreV2 = RestoreV2[dtypes=[DT_FLOAT], _device="/job:localhost/replica:0/task:0/device:CPU:0"](_arg_save/Const_0_0, save/RestoreV2/tensor_names, save/RestoreV2/shape_and_slices)]]

Caused by op 'save/RestoreV2', defined at:
  File "C:\Users\jsh\Anaconda3\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\jsh\Anaconda3\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\jsh\Anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\Users\jsh\Anaconda3\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "C:\Users\jsh\Anaconda3\lib\site-packages\ipykernel\kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
  File "C:\Users\jsh\Anaconda3\lib\site-packages\zmq\eventloop\ioloop.py", line 177, in start
    super(ZMQIOLoop, self)

### Catching the Frozen Layers
#### - 동결된 레이어는 변경되지 않으므로 각 학습 데이터에 대해 가장 상위에 동결된 레이어의 출력을 저장할 수 있음 
- 학습 데이터 당 하나의 동결 레이어를 통과하면 됨으로 속도가 빨라짐 
- 동결레이어로부터 출력의 batch를 생성하여 학습동안에 그것을 데이터에 전달 

In [71]:
reset_graph()

n_inputs = 28 * 28  # MNIST
n_hidden1 = 300 # reused
n_hidden2 = 50  # reused
n_hidden3 = 50  # reused
n_hidden4 = 20  # new!
n_outputs = 10  # new!

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int64, shape=(None), name="y")

with tf.name_scope("dnn"):
    hidden1 = tf.layers.dense(X, n_hidden1, activation=tf.nn.relu,
                              name="hidden1") # reused frozen
    hidden2 = tf.layers.dense(hidden1, n_hidden2, activation=tf.nn.relu,
                              name="hidden2") # reused frozen & cached
    hidden2_stop = tf.stop_gradient(hidden2)
    hidden3 = tf.layers.dense(hidden2_stop, n_hidden3, activation=tf.nn.relu,
                              name="hidden3") # reused, not frozen
    hidden4 = tf.layers.dense(hidden3, n_hidden4, activation=tf.nn.relu,
                              name="hidden4") # new!
    logits = tf.layers.dense(hidden4, n_outputs, name="outputs") # new!

with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss")

with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name="accuracy")

with tf.name_scope("train"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)

In [72]:
reuse_vars = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES,
                               scope="hidden[123]") # regular expression
reuse_vars_dict = dict([(var.op.name, var) for var in reuse_vars])
restore_saver = tf.train.Saver(reuse_vars_dict) # to restore layers 1-3

init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [73]:
import numpy as np

n_batches = mnist.train.num_examples // batch_size

with tf.Session() as sess:
    init.run()
    restore_saver.restore(sess, "./my_model_final.ckpt")
    
    h2_cache = sess.run(hidden2, feed_dict={X: mnist.train.images})
    h2_cache_test = sess.run(hidden2, feed_dict={X: mnist.test.images}) # not shown in the book

    for epoch in range(n_epochs):
        shuffled_idx = np.random.permutation(mnist.train.num_examples)
        hidden2_batches = np.array_split(h2_cache[shuffled_idx], n_batches)
        y_batches = np.array_split(mnist.train.labels[shuffled_idx], n_batches)
        for hidden2_batch, y_batch in zip(hidden2_batches, y_batches):
            sess.run(training_op, feed_dict={hidden2:hidden2_batch, y:y_batch})

        accuracy_val = accuracy.eval(feed_dict={hidden2: h2_cache_test, # not shown
                                                y: mnist.test.labels})  # not shown
        print(epoch, "Test accuracy:", accuracy_val)                    # not shown

    save_path = saver.save(sess, "./my_new_model_final.ckpt")

INFO:tensorflow:Restoring parameters from ./my_model_final.ckpt


NotFoundError: Key hidden1/bias not found in checkpoint
	 [[Node: save/RestoreV2 = RestoreV2[dtypes=[DT_FLOAT], _device="/job:localhost/replica:0/task:0/device:CPU:0"](_arg_save/Const_0_0, save/RestoreV2/tensor_names, save/RestoreV2/shape_and_slices)]]

Caused by op 'save/RestoreV2', defined at:
  File "C:\Users\jsh\Anaconda3\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\jsh\Anaconda3\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\jsh\Anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\Users\jsh\Anaconda3\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "C:\Users\jsh\Anaconda3\lib\site-packages\ipykernel\kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
  File "C:\Users\jsh\Anaconda3\lib\site-packages\zmq\eventloop\ioloop.py", line 177, in start
    super(ZMQIOLoop, self)

### Tweaking, Dropping, or Replacing the Upper Layers
#### - 기존 모델의 출력 레이어는 보통 새로운 작업에 유용하지 않고 원하는 출력 개수와 다를 수도 있기 때문에 대개 교체해서 사용함 
- 비슷하게, 기존 모델의 상위 은닉 레이어는 하위 레이어보다 유용하지 않을 수 있음  
    - 새로운 작업에 있어 가장 유용할 수 있는 고차원 특징이 기존 작업에서 유용하게 쓰인 것과 다를 수 있음 
    
#### - 재사용 할 레이어의 올바른 개수를 찾아야 함 
- 먼저 복사된 모든 레이어를 동결한 후 모델을 학습하고 성능을 확인함
- 그 후 최상위 은닉레이어 중 하나, 두개의 동결을 풀어 성능이 향상되는지 확인 
- 만약 그래도 성능이 좋아지지 않고 사용할 학습데이터가 거의 없다면 
    - 상단의 은닉레이어 한 개, 여러개를 제거하고 나머지 은닉레이어를 모두 동결 
    - 재사용할 레이어를 찾을 때까지 반복 
    
 
- 학습데이터가 충분하면 상위 은닉레이어를 제거하지 않고 교체하거나 추가할 수도 있음 

### Model Zoos
#### - 복잡한 작업을 수행할 떄, 많은 학습 레이블 학습데이터가 없고 유사한 작업으로 학습된 모델도 찾을 수 없을 때 
- 학습 데이터를 더 구할 수 없는 경우 
    - 비지도 선행 학습 수행 
        - 레이블이 없는 학습데이터로 비지도 특징 탐지 알고리즘 (RBM, autoencoder)을 사용하여 학습 
        <img src = "image\ch11\pre.png">

### Pretraining on an Auxiliary Task 
- 레이블된 학습 데이터를 생성하거나 
- 쉽게 얻을 수 있는 보조작업에 대한 첫번째 신경망을 하급 한 후 그 하위 레이어를 실제 작업에 재사용 

- 예)
    - 얼굴 인식 시스템을 만들고 싶다 
    - 하지만 개인에 대한 사진이 몇장 밖에 없음 
    - 임의의 사람들의 사진들을 많이 수집하고 두개의 다른 사진이 동일한 사람을 특징으로 하는지 여부를 감지하는 신경망을 먼저 학습시킴 
    - 첫번째 신경망은 얼굴에 대한 특징을 탐지하는 것을 학습함 
    - 기존 신경망의 하위 레이어를 재사용하여 얼굴 분류기 신경망을 학습 
    
    
- 레이블이 없는 데이터에 레이블을 지정하는 것은 비용이 많이 소요됨 
    - 모든 학습 예제를 'good'레이블로 지정
    - 그 예제를 손상시켜 새로운 학습 데이터를 생성하고 'bad'레이블을 줌 
    - 첫번째 신경망이 good, bad 레이블을 분류하도록 학습
    
    
- 첫번재 신경망을 각 학습 데이터에 대한 점수를 출력하도록 학습 
    - 좋은 데이터의 점수가 나쁜 데이터의 점수보다 적어도 약간의 margin 만큼 큰지에 대해 확인하는 비용함수 사용 
    - max margin learning 